In [ ]:
1 + 1

In [ ]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

In [ ]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [ ]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules
using PyCall
pd = pyimport("pandas")

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("network.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "spring.jl"))
    includef(srcdir("scenarios", "fall.jl"))
end

Report results on FALL

In [ ]:
function report_phys101(dataset; verbose=false)
    dataset = "phys101/$dataset"
    df = pd.DataFrame(
        columns=("Method", "Normliased RMSE (train)", "Normliased RMSE (test)", "EM seed", "Data seed")
    )

    count = 1
    for seed in 1:3, shuffleseed in 1:6
        hps = (ntrains=2, seed=seed, shuffleseed=shuffleseed)

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 2; shuffleseed=hps.shuffleseed, istrain=false)

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "em.jld2"))
        catch
            println("seed $(hps.seed) unfinished")
        end

        @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, trace = res

        for (force, method) in [(trace[end].force, "BSP"), (ZeroForce(), "Zero")]
            verbose && println("count: $count")
            verbose && if method == "BSP"
                println(BayesianSymbolic.get_executable(force.tree, force.grammar))
                println(force.constant)
            end
            evalres = evalforce(ScenarioModel, scenarios, attributes, force, ealg, elike, mlike)
            evalres_test = evalforce(ScenarioModel, scenarios_test, attributes_test, force, ealg, elike, mlike)

            # Add to data frame
            df.loc[count] = [method, evalres.normrmse, evalres_test.normrmse, seed, shuffleseed]
            count = count + 1
        end
        verbose && println()
    end
    
    return df
end

Report results on FALL

In [ ]:
df = report_phys101("fall"; verbose=true)

In [ ]:
df.groupby(["Method"]).median()

In [ ]:
df.groupby(["Method"]).quantile([.25, .75])

In [ ]:
df.groupby(["Method"]).mean()

In [ ]:
df.groupby(["Method"]).std()

Report results on SPRING

In [ ]:
df = report_phys101("spring"; verbose=true)

In [ ]:
df.groupby(["Method"]).median()

In [ ]:
df.groupby(["Method"]).quantile([.25, .75])

In [ ]:
df.groupby(["Method"]).mean()

In [ ]:
df.groupby(["Method"]).std()